使用discord机器人实现推文获取总结

In [1]:
!pip install discord.py nest_asyncio asyncio googletrans==4.0.0-rc1 pandas openai snscrape pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 3

In [12]:
import os
import discord
from discord.ext import commands
import snscrape.modules.twitter as twitter
import pandas as pd
import openai
import logging
from googletrans import Translator
import nest_asyncio
from datetime import datetime, timedelta
from PIL import Image
import io
import requests

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

openai.api_key = ""

def scrape_and_summarize():
    
    # three_days_ago = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
    query = " AI (use OR uses OR using OR example OR examples OR scenario OR scenarios OR application OR applications)" 

    results_to_retrieve = 10
    logger.info(f"Searching for '{query}' on Twitter")

    scraper = twitter.TwitterSearchScraper(query)
    tweets = sorted(scraper.get_items(), key=lambda x: x.likeCount + x.quoteCount + x.replyCount, reverse=True)
    print("tweets count",len(tweets))
    translator = Translator()
    tweet_data = []

    for i, tweet in enumerate(tweets[:results_to_retrieve]):
        tweet_url = tweet.url
        tweet_content = tweet.content

        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=f"Please summarize the following tweet:\n\n{tweet_content}\n\nSummary:",
            max_tokens=30,
            n=1,
            stop=None,
            temperature=0.7,
        )

        summary = response.choices[0].text.strip()
        translated_summary = translator.translate(summary, dest="zh-cn").text
        print("here is summary ",translated_summary);
        tweet_data.append((tweet_url, tweet_content, summary, translated_summary))
        
    return tweet_data
    # df = pd.DataFrame(tweet_data, columns=["Tweet URL", "Tweet Content", "Summary", "Translated Summary (Chinese)"])
    # df.to_csv("tweet_summaries.csv", index=False)

# Discord bot code
TOKEN = ""

intents = discord.Intents.default()
intents.messages = True
intents.guilds = True
intents.reactions = True
intents.message_content = True  # 添加这一行
bot = commands.Bot(command_prefix='/', intents=intents)

@bot.event
async def on_message(message):
    # Ignore messages sent by the bot itself
    if message.author == bot.user:
        return

    # Check if the message contains "vvink"
    if "vvink" in message.content.lower():
        await message.channel.send("yyds")

    # Process commands
    await bot.process_commands(message)

@bot.event
async def on_ready():
    print(f'{bot.user} has connected to Discord!')

@bot.command(name='news', help='Returns summaries of the top tweets about ChatGPT-4')
async def news(ctx):
    print("hello bot")
    processing_message = await ctx.send(f'{ctx.author.mention}，正在处理您的请求，请稍候...')
    tweet_summaries = scrape_and_summarize()
    await processing_message.edit(content="处理完成！正在发送推文总结...")

    for url, content, summary, translated_summary in tweet_summaries:
        response = f"**Tweet 地址**: {url}\n **Tweet 总结**: {translated_summary}\n"
        await ctx.send(response)

    await ctx.send(response)

@bot.command(name='clip', help='Crops an image into 4 smaller images with the specified aspect ratio. Default aspect ratio is 1:1 if not provided.')
async def clip(ctx, image_url: str, aspect_ratio: str = '1:1'):
    width_ratio, height_ratio = map(int, aspect_ratio.split(':'))
    if width_ratio <= 0 or height_ratio <= 0:
        await ctx.send("Width and height ratio must be greater than 0.")
        return

    processing_message = await ctx.send(f'{ctx.author.mention}，正在处理您的请求，请稍候...')

    # Download the image
    img_data = requests.get(image_url).content
    image = Image.open(io.BytesIO(img_data))

    # Crop the image into four smaller images with the specified aspect ratio
    width, height = image.size
    clipped_width = width // 2
    clipped_height = height // 2

    cropped_images = [
        image.crop((0, 0, clipped_width, clipped_height)),
        image.crop((clipped_width, 0, width, clipped_height)),
        image.crop((0, clipped_height, clipped_width, height)),
        image.crop((clipped_width, clipped_height, width, height)),
    ]

    # Send the cropped images
    for index, cropped_image in enumerate(cropped_images):
        img_byte_array = io.BytesIO()
        cropped_image.save(img_byte_array, format="PNG")
        img_byte_array.seek(0)
        file = discord.File(img_byte_array, filename=f"cropped_image_{index + 1}.png")
        await ctx.send(file=file)

    await processing_message.edit(content=f'{ctx.author.mention}，处理完毕，已发送裁剪后的图片。')

nest_asyncio.apply()
bot.run(TOKEN)


2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
2023-03-31 02:56:03 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2023-03-31 02:56:04 INFO     discord.gateway Shard ID None has

InsaneMind#8440 has connected to Discord!
